In [ ]:
"""
NOTEBOOK 05: ANÁLISIS DEL VIDEO SEGMENTADO
============================================
Aplicar segmentación automática a frames de video y validar 
contra anotaciones manuales del notebook 03.

Tareas:
1. Cargar y procesar frames segmentados
2. Extraer métricas de segmentación automática
3. Integrar con anotaciones manuales
4. Calcular errores de validación

Entrada: Frames segmentados (notebook 04), anotaciones (notebook 03)
Salida: CSV de integración automático vs. manual, matriz de validación

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from scipy import ndimage, stats

# Configuración de visualización
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10

In [ ]:
"""
TAREA 2: Integración de Resultados Automáticos vs. Manuales
==========================================================
Crea matriz de validación cruzada para evaluar precisión
del algoritmo de segmentación.

Comparativas:
- SVD manual vs. densidad automática
- Categorías de flujo vs. componentes detectados
- Correlación entre métricas

Métricas de validación:
- Error absoluto medio (MAE)
- Correlación de Spearman
- Acuerdo visual-automático
"""

print("\n" + "=" * 70)
print("TAREA 2: INTEGRACIÓN DE SEGMENTACIÓN Y ANOTACIONES")
print("=" * 70)

# Crear matriz de integración
integration_results = []

for idx, row in df_annotations.iterrows():
    record_id = row['record_id']
    patient = row['patient']
    
    result = {
        'record_id': record_id,
        'patient': patient,
        'visit_code': row['visit_code'],
        
        # Datos manuales (anotaciones)
        'manual_total_vessels': row['total_vessels'],
        'manual_svd': row['svd'],
        'manual_tvd': row['tvd'],
        'manual_cat3': row['count_cat3'],
    }
    
    # Buscar video segmentado coincidente
    matching_video = None
    for video_name in video_metrics.keys():
        # Matching simple: si el nombre del paciente está en el video
        if patient.lower() in video_name.lower():
            matching_video = video_name
            break
    
    if matching_video and matching_video in video_metrics:
        metrics = video_metrics[matching_video]
        result['computed_density_mean'] = metrics['mean_density']
        result['computed_components_mean'] = metrics['mean_components']
        result['computed_skeleton_length'] = metrics['mean_skeleton_length']
        result['matched'] = True
        
        # Calcular diferencias
        result['density_error'] = abs(result['manual_svd'] * 100 - metrics['mean_density'])
        result['component_ratio'] = metrics['mean_components'] / max(result['manual_total_vessels'], 1)
    else:
        result['computed_density_mean'] = np.nan
        result['computed_components_mean'] = np.nan
        result['computed_skeleton_length'] = np.nan
        result['matched'] = False
        result['density_error'] = np.nan
        result['component_ratio'] = np.nan
    
    integration_results.append(result)

# Crear DataFrame de integración
df_integration = pd.DataFrame(integration_results)

print("\nMatriz de validación cruzada (primeros 10 registros):")
print("=" * 70)
display_cols = ['record_id', 'patient', 'manual_svd', 'computed_density_mean', 
                'manual_total_vessels', 'computed_components_mean']
print(df_integration[display_cols].head(10).to_string())

# Estadísticas de validación
print("\n\nESTADÍSTICAS DE VALIDACIÓN:")
print("=" * 70)

matched = df_integration[df_integration['matched'] == True]
if len(matched) > 0:
    print(f"\nRegistros coincidentes: {len(matched)} de {len(df_integration)}")
    print(f"  Error de densidad (MAE): {matched['density_error'].mean():.3f}")
    print(f"  Std error densidad: {matched['density_error'].std():.3f}")
    
    print(f"\n  Ratio componentes:")
    print(f"    Media: {matched['component_ratio'].mean():.2f}")
    print(f"    Std: {matched['component_ratio'].std():.2f}")
    print(f"    Rango: {matched['component_ratio'].min():.2f} - {matched['component_ratio'].max():.2f}")
else:
    print("⚠️ No hay registros coincidentes entre segmentación y anotaciones")

# Guardar resultados de integración
output_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/segmentation_annotation_integration.csv')
df_integration.to_csv(output_path, index=False)

print(f"\n✓ Integración guardada en: {output_path}")
print(f"  Total de registros: {len(df_integration)}")
print(f"  Registros válidos: {len(matched)}")

print("\n" + "=" * 70)
print("✓ NOTEBOOK 05 COMPLETADO")
print("=" * 70)
print("\nReady for Notebook 06: Feature Extraction (Space-Time Analysis)")

In [ ]:
"""
TAREA 1: Cargar Anotaciones y Frames Segmentados
================================================
Carga dos fuentes de datos:
1. Anotaciones manuales (notebook 03) - Ground truth
2. Frames segmentados (notebook 04) - Salida automática

Objetivo: Crear matriz de validación cruzada
"""

print("\n" + "=" * 70)
print("TAREA 1: CARGA DE DATOS - ANOTACIONES Y SEGMENTACIÓN")
print("=" * 70)

# Cargar anotaciones procesadas del notebook 03
annotations_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')

if annotations_path.exists():
    df_annotations = pd.read_csv(annotations_path)
    print(f"\n✓ Anotaciones cargadas: {len(df_annotations)} registros")
    print(f"  Columnas clave: {list(df_annotations.columns[:8])}")
else:
    print(f"\n⚠️ Archivo no encontrado: {annotations_path}")
    # Crear datos de demostración
    df_annotations = pd.DataFrame({
        'record_id': [f'patient{i}_D1-V1' for i in range(3)],
        'patient': [f'patient{i}' for i in range(3)],
        'visit_code': ['D1-V1', 'D1-V1', 'D1-V1'],
        'total_vessels': [25, 30, 22],
        'svd': [0.45, 0.52, 0.38],
        'tvd': [0.78, 0.85, 0.72],
        'count_cat3': [18, 22, 16]
    })
    print(f"\n✓ Anotaciones de demostración creadas: {len(df_annotations)} registros")

# Cargar frames segmentados del notebook 04
segmented_dir = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/segmented')

video_metrics = {}

print(f"\nBuscando frames segmentados en: {segmented_dir}")

if segmented_dir.exists():
    for video_dir in sorted(segmented_dir.iterdir()):
        if video_dir.is_dir():
            frames_paths = sorted(video_dir.glob("segmented_frame_*.png"))
            
            if frames_paths:
                frame_metrics = []
                
                # Procesar cada frame segmentado
                for frame_idx, frame_path in enumerate(frames_paths):
                    frame = cv2.imread(str(frame_path), cv2.IMREAD_GRAYSCALE)
                    if frame is not None:
                        # Computar métricas de segmentación
                        total_pixels = frame.size
                        vessel_pixels = np.sum(frame > 0)
                        
                        # Análisis esqueletal
                        skeleton = morphology.skeletonize(frame / 255)
                        skeleton_length = np.sum(skeleton)
                        
                        # Análisis de conectividad
                        _, num_components = cv2.connectedComponents(frame)
                        
                        metrics = {
                            'frame_idx': frame_idx,
                            'vessel_density': 100 * vessel_pixels / total_pixels,
                            'vessel_pixels': vessel_pixels,
                            'num_components': num_components - 1,  # Excluir background
                            'skeleton_length': skeleton_length
                        }
                        frame_metrics.append(metrics)
                
                # Estadísticas por video
                df_metrics = pd.DataFrame(frame_metrics)
                video_metrics[video_dir.name] = {
                    'n_frames': len(frame_metrics),
                    'mean_density': df_metrics['vessel_density'].mean(),
                    'std_density': df_metrics['vessel_density'].std(),
                    'mean_components': df_metrics['num_components'].mean(),
                    'mean_skeleton_length': df_metrics['skeleton_length'].mean(),
                    'frame_metrics': df_metrics
                }
                
                print(f"  ✓ {video_dir.name}: {len(frame_metrics)} frames analizados")
else:
    print(f"  ⚠️ Directorio no encontrado")

print(f"\n✓ Total de videos segmentados: {len(video_metrics)}")

# Mostrar resumen de métricas
print("\nRESUMEN DE MÉTRICAS SEGMENTADAS:")
print("=" * 70)
print(f"{'Video':<20} {'Frames':>8} {'Densidad%':>12} {'Componentes':>12}")
print("=" * 70)

for video_name, metrics in video_metrics.items():
    print(f"{video_name:<20} {metrics['n_frames']:>8d} "
          f"{metrics['mean_density']:>11.2f}% {metrics['mean_components']:>12.1f}")

In [ ]:
"""
NOTEBOOK 05: ANÁLISIS DEL VIDEO SEGMENTADO Y VALIDACIÓN
========================================================
Objetivo: Aplicar máscaras de segmentación a frames de video y extraer 
métricas que pueden compararse con anotaciones manuales del notebook 03.

Validación cruzada:
- Segmentación automática → Características extraídas
- Anotaciones manuales → Referencia de calidad  
- Comparar y evaluar precisión del método automático

Salidas:
- Métricas por video segmentado
- Integración de resultados automáticos vs. manuales
- Matriz de validación cruzada

Nota: Este paso es crítico para calibración del algoritmo
"""

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from skimage import morphology
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

print("=" * 70)
print("ETAPA 05: ANÁLISIS DEL VIDEO SEGMENTADO")
print("=" * 70)

# Notebook 05: Análisis del Video Segmentado

**Objetivo:** Aplicar segmentación automática y validar contra anotaciones manuales

**Entrada:**
- Anotaciones manuales (notebook 03)
- Frames segmentados (notebook 04)

**Proceso:**
1. Cargar y procesarframes segmentados
2. Extraer métricas de segmentación
3. Integrar con anotaciones manuales
4. Calcular error y validación cruzada

**Salida:**
- CSV de integración (automático vs. manual)
- Estadísticas de validación
- Matriz de error para ajuste de algoritmo

# 05 - Análisis del video segmentado

## Objetivo
- Aplicar segmentador a todos los frames
- Generar imagen espacio-tiempo por capilar


In [ ]:
from src.features.space_time_diagram import build_space_time_image


# Procesamiento de un video
video_path = "../data/processed/video1"
model.eval()
space_time_img = build_space_time_image(video_path, model)
plt.imshow(space_time_img, cmap='gray')
plt.title("Imagen espacio-tiempo")
